In [2]:
import sklearn
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [3]:
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
log_clf = LogisticRegression(solver='lbfgs', random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=150, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42, probability=True)

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [5]:
def build_nn():
    model = Sequential([Dense(50, activation='relu', input_shape=[2]),Dense(1, activation='sigmoid')])
    model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [9]:
pip install scikeras

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for scikeras from https://files.pythonhosted.org/packages/ea/09/1c02aa24daf7a003c06f629fbb69dc9ae1bda1b247d7b8981e550d752ac9/scikeras-0.13.0-py3-none-any.whl.metadata
  Obtaining dependency information for scikit-learn>=1.4.2 from https://files.pythonhosted.org/packages/a1/a6/c5b78606743a1f28eae8f11973de6613a5ee87366796583fb74c67d54939/scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/32/d5/f9a850d79b0851d1d4ef6456097579a9005b31fea68726a4ae5f2d82ddd9/threadpoolctl-3.6.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.1 MB 3.5 MB/s eta 0:00:04
   -- ------------------------------------- 0.8/11.1 MB 10.1 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.1

In [10]:
from scikeras.wrappers import KerasClassifier
keras_clf = KerasClassifier(model=build_nn, epochs=500, verbose=0)

In [11]:
keras_clf._estimator_type = 'classifier'

In [13]:
voting = VotingClassifier(estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf),('keras',keras_clf)],
                                     voting='soft',flatten_transform=True)

In [14]:
voting.fit(X_train, y_train)

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf',
                              RandomForestClassifier(n_estimators=150,
                                                     random_state=42)),
                             ('svc', SVC(probability=True, random_state=42)),
                             ('keras',
                              KerasClassifier(epochs=500, model=<function build_nn at 0x00000269648BD940>, verbose=0))],
                 voting='soft')

In [16]:
for clf in (log_clf,rnd_clf,svm_clf,keras_clf,voting):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.904
SVC 0.896


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KerasClassifier 0.872


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


VotingClassifier 0.912
